In [3]:
from gurobipy import *


In [4]:
milp_model = Model("milp")

Restricted license - for non-production use only - expires 2027-11-29


In [7]:
# # Gurobi WLS license file
# # Your credentials are private and should not be shared or copied to public repositories.
# # Visit https://license.gurobi.com/manager/doc/overview for more information.
# WLSACCESSID=5567d935-0c99-4630-877a-2f0faa1e1d46
# WLSSECRET=306c2296-f7d9-4eec-aab4-8966fdf8d9ac
# LICENSEID=2725120


import gurobipy as gp

# <<< REPLACE WITH YOUR REAL VALUES (as strings) >>>
ACCESS_ID = "d06a70a9-a575-46f5-b3d3-df24ed75f420"   # string!
SECRET    = "4028d0de-05b9-4ab2-8917-4ee328c3a7b9"   # string!
LICENSEID = 2783276                                # string is safest

# Quick type check (optional)
assert isinstance(ACCESS_ID, str) and isinstance(SECRET, str) and isinstance(LICENSEID, (str, int))

env = gp.Env(empty=True)
env.setParam("WLSACCESSID", str(ACCESS_ID))
env.setParam("WLSSECRET",   str(SECRET))
env.setParam("LICENSEID",   LICENSEID)   # pass as string to be safe
# env.setParam("WLSSERVER", "https://license.gurobi.com")  # optional
env.start()

# smoke test
m = gp.Model("ping", env=env)
x = m.addVar(ub=1.0)
m.setObjective(x, gp.GRB.MAXIMIZE)
m.optimize()
print("Gurobi OK, x =", x.X)


Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2783276
Academic license 2783276 - for non-commercial use only - registered to ro___@gmail.com
Gurobi Optimizer version 13.0.1 build v13.0.1rc0 (mac64[arm] - Darwin 23.0.0 23A344)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Academic license 2783276 - for non-commercial use only - registered to ro___@gmail.com
Optimize a model with 0 rows, 1 columns and 0 nonzeros (Max)
Model fingerprint: 0x5939e866
Model has 1 linear objective coefficients
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [0e+00, 0e+00]

Presolve removed 0 rows and 1 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01

In [8]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

"""
Full MILP toolkit (Gurobi) for:
  (A) XOR-differentials (your original style)
  (B) Restricted rotational XOR-differentials: enforce Δ = rot(Δ,k)
  (C) Linear trails (Fu-style add model + correct fan-out constraints)

NOTE:
- This is a *bit-level* model. It grows fast.
- Start with nbits=8/16 and small rounds to sanity-check.
"""

import re
from typing import Dict, Tuple, Optional, Iterable, List
import gurobipy as gp


# ----------------- bit helpers -----------------
def rotl(bits, r: int):
    n = len(bits); r %= n
    return [bits[(i - r) % n] for i in range(n)]

def rotr(bits, r: int):
    n = len(bits); r %= n
    return [bits[(i + r) % n] for i in range(n)]

def bits_from_word(m: gp.Model, name: str, nbits: int):
    return [m.addVar(vtype=gp.GRB.BINARY, name=f"{name}[{i}]") for i in range(nbits)]

def eq_word(m: gp.Model, a, b, name: str):
    """Enforce a == b bitwise."""
    assert len(a) == len(b)
    for i in range(len(a)):
        m.addConstr(a[i] == b[i], name=f"{name}[{i}]")


# ============================================================
#  XOR-differential primitives (your style)
# ============================================================
def add_word(m: gp.Model, x, y, name: str):
    """Mod-2^n add encoded exactly per bit: x + y + cin = z + 2*cout, cin=0."""
    n = len(x)
    z = [m.addVar(vtype=gp.GRB.BINARY, name=f"{name}_z[{i}]") for i in range(n)]
    c = [m.addVar(vtype=gp.GRB.BINARY, name=f"{name}_c[{i}]") for i in range(n+1)]
    m.addConstr(c[0] == 0, name=f"{name}_cin0")
    for i in range(n):
        m.addConstr(x[i] + y[i] + c[i] == z[i] + 2*c[i+1], name=f"{name}_fa[{i}]")
    return z, c[1:]  # carry-outs c[1]..c[n] (c[n] is overflow)

def xor2_word(m: gp.Model, a, b, name: str):
    """Bitwise XOR via linearization: a + b = z + 2*(a&b)."""
    n = len(a)
    z = [m.addVar(vtype=gp.GRB.BINARY, name=f"{name}_z[{i}]") for i in range(n)]
    t = [m.addVar(vtype=gp.GRB.BINARY, name=f"{name}_t[{i}]") for i in range(n)]
    for i in range(n):
        m.addConstr(t[i] <= a[i]); m.addConstr(t[i] <= b[i]); m.addConstr(t[i] >= a[i] + b[i] - 1)
        m.addConstr(a[i] + b[i] == z[i] + 2*t[i], name=f"{name}_xor[{i}]")
    return z


def build_fha_xordiff_milp(
    rounds: int = 6,
    nbits: int = 64,
    lanes: int = 8,
    ROT_A: Optional[Iterable[int]] = None,
    ROT_B: Optional[Iterable[int]] = None,
    ROT_G: Optional[Iterable[int]] = None,
    iterative: bool = True,
    pins: Optional[Dict[str, int]] = None,
    mode: str = "min",                   # 'min' | 'feas_ge' | 'feas_le'
    target_weight: int = 255,            # used by feas modes
    cube_t0_bits: int = 0,
    cube_t0_value: int = 0,
    threads: int = 1,
    timelimit: Optional[int] = None,
    env: Optional[gp.Env] = None,
) -> Tuple[gp.Model, gp.LinExpr]:
    """
    XOR-differential MILP. Weight = sum of carry-outs from Mix-1 adds (overflow excluded).
    """
    if ROT_A is None: ROT_A = [63,19,39,15,23,31,11,5]
    if ROT_B is None: ROT_B = [21, 3,31,33,45, 5,17,7]
    if ROT_G is None: ROT_G = [25,45,51, 5,63,47, 7,1]
    ROT_A = list(ROT_A); ROT_B = list(ROT_B); ROT_G = list(ROT_G)
    if len(ROT_A) < lanes: ROT_A = (ROT_A * ((lanes + len(ROT_A)-1)//len(ROT_A)))[:lanes]
    if len(ROT_B) < lanes: ROT_B = (ROT_B * ((lanes + len(ROT_B)-1)//len(ROT_B)))[:lanes]
    if len(ROT_G) < lanes: ROT_G = (ROT_G * ((lanes + len(ROT_G)-1)//len(ROT_G)))[:lanes]

    m = gp.Model("fha_xordiff", env=env) if env else gp.Model("fha_xordiff")
    if threads:   m.Params.Threads   = threads
    if timelimit: m.Params.TimeLimit = timelimit

    L = [[bits_from_word(m, f"L{j}_{i}", nbits) for i in range(rounds+1)] for j in range(lanes)]
    R = [[bits_from_word(m, f"R{j}_{i}", nbits) for i in range(rounds+1)] for j in range(lanes)]

    weight_bits: List[gp.Var] = []

    for i in range(rounds):
        # 1) L_{i+1} = R_i
        for j in range(lanes):
            for b in range(nbits):
                m.addConstr(L[j][i+1][b] == R[j][i][b], name=f"copy_L[{j},{i},{b}]")

        # 2) F(R_i)
        V1, V2, V3 = [[] for _ in range(lanes)], [[] for _ in range(lanes)], [[] for _ in range(lanes)]
        for j in range(lanes):
            j1, j2 = (j+1) % lanes, (j+2) % lanes
            z1, carries = add_word(m, R[j][i], rotl(R[j1][i], ROT_A[j] % nbits), f"add_{i}_{j}")
            V1[j] = z1
            if carries:
                weight_bits += carries[:-1]  # exclude overflow carry
            V2[j] = xor2_word(m, V1[j], rotr(R[j2][i], ROT_B[j] % nbits), f"xor2_{i}_{j}")
            V3[j] = V2[j]  # RC dropped (conservative in XOR-diff)

        # t = XOR over lanes of V3[j]
        t = V3[0]
        for j in range(1, lanes):
            t = xor2_word(m, t, V3[j], f"tacc_{i}_{j}")

        if i == 0 and cube_t0_bits > 0:
            for b in range(cube_t0_bits):
                bitval = (cube_t0_value >> b) & 1
                t[b].LB = bitval
                t[b].UB = bitval

        # fo[j] = v3[j] XOR ROTL(t, ROT_G[j])
        FO = [None] * lanes
        for j in range(lanes):
            FO[j] = xor2_word(m, V3[j], rotl(t, ROT_G[j] % nbits), f"fo_{i}_{j}")

        # 3) R_{i+1} = L_i XOR FO
        for j in range(lanes):
            R[j][i+1] = xor2_word(m, L[j][i], FO[j], f"Rnext_{i}_{j}")

    if iterative:
        for j in range(lanes):
            for b in range(nbits):
                m.addConstr(L[j][0][b] == L[j][rounds][b], name=f"iter_L[{j},{b}]")
                m.addConstr(R[j][0][b] == R[j][rounds][b], name=f"iter_R[{j},{b}]")

    if pins:
        for key, val in pins.items():
            mo = re.fullmatch(r'([LR])(\d+)_(\d+)', key)
            if not mo:
                raise ValueError(f"Bad pin key: {key}")
            side, jj, rnd = mo.group(1), int(mo.group(2)), int(mo.group(3))
            word = L[jj][rnd] if side == 'L' else R[jj][rnd]
            for b in range(nbits):
                bit = (val >> b) & 1
                word[b].LB = bit
                word[b].UB = bit

    W = gp.quicksum(weight_bits)

    if mode == "min":
        m.setObjective(W, gp.GRB.MINIMIZE)
    elif mode == "feas_ge":
        m.addConstr(W >= target_weight, name="W_ge")
        m.setObjective(0, gp.GRB.MINIMIZE)
    elif mode == "feas_le":
        m.addConstr(W <= target_weight, name="W_le")
        m.setObjective(0, gp.GRB.MINIMIZE)
    else:
        raise ValueError("mode must be 'min' | 'feas_ge' | 'feas_le'")

    return m, W



#  Restricted rotational XOR-differential check

def add_rot_invariance_word(m: gp.Model, word_bits, k: int, name: str):
    """Enforce word_bits == rotl(word_bits, k)."""
    n = len(word_bits); k %= n
    for b in range(n):
        m.addConstr(word_bits[b] == word_bits[(b - k) % n], name=f"{name}[{b}]")

def build_rotational_xordiff(
    rounds: int,
    k: int,
    nbits: int = 64,
    lanes: int = 8,
    constrain_rounds: Iterable[int] = (0,),
    **kwargs
) -> Tuple[gp.Model, gp.LinExpr]:
 
    m, W = build_fha_xordiff_milp(rounds=rounds, nbits=nbits, lanes=lanes, mode="min", **kwargs)

    for rnd in constrain_rounds:
        for j in range(lanes):
            Lw = [m.getVarByName(f"L{j}_{rnd}[{b}]") for b in range(nbits)]
            Rw = [m.getVarByName(f"R{j}_{rnd}[{b}]") for b in range(nbits)]
            add_rot_invariance_word(m, Lw, k, name=f"rotInv_L{j}_{rnd}")
            add_rot_invariance_word(m, Rw, k, name=f"rotInv_R{j}_{rnd}")

    # Non-trivial diff
    L0sum = gp.quicksum(m.getVarByName(f"L{j}_0[{b}]") for j in range(lanes) for b in range(nbits))
    R0sum = gp.quicksum(m.getVarByName(f"R{j}_0[{b}]") for j in range(lanes) for b in range(nbits))
    m.addConstr(L0sum + R0sum >= 1, name="nontrivial_rot")

    m.setObjective(W, gp.GRB.MINIMIZE)
    return m, W



#  Linear trail MILP (Fu-style addition + correct fan-out)


def xor3_parity0_bit(m: gp.Model, a, b, c, name: str):
    """
    Enforce even parity: a ⊕ b ⊕ c = 0  (equivalently c = a ⊕ b).
    Allowed sums: a+b+c ∈ {0,2}.
    """
    d = m.addVar(vtype=gp.GRB.BINARY, name=f"{name}_d")
    m.addConstr(d >= a, name=f"{name}_dgea")
    m.addConstr(d >= b, name=f"{name}_dgeb")
    m.addConstr(d >= c, name=f"{name}_dgec")
    m.addConstr(a + b + c >= 2*d, name=f"{name}_sumge2d")
    m.addConstr(a + b + c <= 2,   name=f"{name}_sumle2")
    return d

def xor_reduce_word(m: gp.Model, inputs, name: str):
    """out = XOR of all input words (bitwise)."""
    nbits = len(inputs[0])
    out = [m.addVar(vtype=gp.GRB.BINARY, name=f"{name}_out[{i}]") for i in range(nbits)]
    if len(inputs) == 1:
        for i in range(nbits):
            m.addConstr(out[i] == inputs[0][i], name=f"{name}_copy[{i}]")
        return out

    cur = inputs[0]
    for k in range(1, len(inputs)):
        nxt = [m.addVar(vtype=gp.GRB.BINARY, name=f"{name}_t{k}[{i}]") for i in range(nbits)]
        for i in range(nbits):
            xor3_parity0_bit(m, cur[i], inputs[k][i], nxt[i], name=f"{name}_xor{k}[{i}]")
        cur = nxt

    for i in range(nbits):
        m.addConstr(out[i] == cur[i], name=f"{name}_outEq[{i}]")
    return out

def fanout_conservation_word(m: gp.Model, node_mask, use_masks, name: str):
    """
    Fan-out constraint for linear trails (mask conservation):
      node_mask = XOR(use_masks[0], use_masks[1], ...).
    """
    xsum = xor_reduce_word(m, use_masks, name=f"{name}_xsum")
    for i in range(len(node_mask)):
        m.addConstr(node_mask[i] == xsum[i], name=f"{name}_cons[{i}]")


def add_linear_addition_fu(
    m: gp.Model,
    gamma,  # output mask Γ
    la,     # input mask Λa
    lb,     # input mask Λb
    name: str,
):
    """
    Fu-style ARX addition linear model: z = x + y mod 2^n.
    Weight = sum_{i=1..n-1} s[i].
    """
    n = len(gamma)
    s = [m.addVar(vtype=gp.GRB.BINARY, name=f"{name}_s[{i}]") for i in range(n + 1)]
    m.addConstr(s[n] == 0, name=f"{name}_sn0")

    for i in range(n):
        m.addConstr(s[i+1] - gamma[i] - la[i] + lb[i] + s[i] >= 0, name=f"{name}_c0[{i}]")
        m.addConstr(s[i+1] + gamma[i] + la[i] - lb[i] - s[i] >= 0, name=f"{name}_c1[{i}]")
        m.addConstr(s[i+1] + gamma[i] - la[i] - lb[i] + s[i] >= 0, name=f"{name}_c2[{i}]")
        m.addConstr(s[i+1] - gamma[i] + la[i] - lb[i] + s[i] >= 0, name=f"{name}_c3[{i}]")
        m.addConstr(s[i+1] + gamma[i] - la[i] + lb[i] - s[i] >= 0, name=f"{name}_c4[{i}]")
        m.addConstr(s[i+1] - gamma[i] + la[i] + lb[i] - s[i] >= 0, name=f"{name}_c5[{i}]")
        m.addConstr(-s[i+1] + gamma[i] + la[i] + lb[i] + s[i] >= 0, name=f"{name}_c6[{i}]")
        m.addConstr(s[i+1] + gamma[i] + la[i] + lb[i] + s[i] <= 4, name=f"{name}_c7[{i}]")

    w = gp.quicksum(s[i] for i in range(1, n))
    return w, s


def build_fha_linear_milp(
    rounds: int = 6,
    nbits: int = 64,
    lanes: int = 8,
    ROT_A: Optional[Iterable[int]] = None,
    ROT_B: Optional[Iterable[int]] = None,
    ROT_G: Optional[Iterable[int]] = None,
    pins: Optional[Dict[str, int]] = None,
    threads: int = 1,
    timelimit: Optional[int] = None,
    env: Optional[gp.Env] = None,
) -> Tuple[gp.Model, gp.LinExpr]:
 

    if ROT_A is None: ROT_A = [63,19,39,15,23,31,11,5]
    if ROT_B is None: ROT_B = [21, 3,31,33,45, 5,17,7]
    if ROT_G is None: ROT_G = [25,45,51, 5,63,47, 7,1]
    ROT_A = list(ROT_A); ROT_B = list(ROT_B); ROT_G = list(ROT_G)
    if len(ROT_A) < lanes: ROT_A = (ROT_A * ((lanes + len(ROT_A)-1)//len(ROT_A)))[:lanes]
    if len(ROT_B) < lanes: ROT_B = (ROT_B * ((lanes + len(ROT_B)-1)//len(ROT_B)))[:lanes]
    if len(ROT_G) < lanes: ROT_G = (ROT_G * ((lanes + len(ROT_G)-1)//len(ROT_G)))[:lanes]

    m = gp.Model("fha_linear", env=env) if env else gp.Model("fha_linear")
    if threads:   m.Params.Threads   = threads
    if timelimit: m.Params.TimeLimit = timelimit

    # State masks
    Lm = [[bits_from_word(m, f"Lm{j}_{i}", nbits) for i in range(rounds+1)] for j in range(lanes)]
    Rm = [[bits_from_word(m, f"Rm{j}_{i}", nbits) for i in range(rounds+1)] for j in range(lanes)]

    W_terms: List[gp.LinExpr] = []

    for i in range(rounds):
        # --- Fan-out split for each Rm[j][i] (used in multiple places) ---
        Rcopy = [bits_from_word(m, f"Rcopy_{i}_{j}", nbits) for j in range(lanes)]
        Radd  = [bits_from_word(m, f"Radd_{i}_{j}",  nbits) for j in range(lanes)]
        RrotA = [bits_from_word(m, f"RrotA_{i}_{j}", nbits) for j in range(lanes)]
        RrotB = [bits_from_word(m, f"RrotB_{i}_{j}", nbits) for j in range(lanes)]
        for j in range(lanes):
            fanout_conservation_word(m, Rm[j][i], [Rcopy[j], Radd[j], RrotA[j], RrotB[j]], name=f"fan_R_{i}_{j}")

        # 1) L_{i+1} = R_i  (copy use)
        for j in range(lanes):
            eq_word(m, Lm[j][i+1], Rcopy[j], name=f"copyLm_{j}_{i}")

        # 2) Mix-1: v1 = Radd[j] + ROTL(RrotA[j1], ROT_A[j])
        V1m = [bits_from_word(m, f"V1m_{i}_{j}", nbits) for j in range(lanes)]
        for j in range(lanes):
            j1 = (j + 1) % lanes
            la = Radd[j]
            lb = rotl(RrotA[j1], ROT_A[j] % nbits)
            gamma = V1m[j]
            w_add, _ = add_linear_addition_fu(m, gamma=gamma, la=la, lb=lb, name=f"linadd_{i}_{j}")
            W_terms.append(w_add)

        # 3) Mix-2: v2 = v1 XOR ROTR(RrotB[j2], ROT_B[j])
        V2m = [bits_from_word(m, f"V2m_{i}_{j}", nbits) for j in range(lanes)]
        for j in range(lanes):
            j2 = (j + 2) % lanes
            # XOR: for z = x XOR y, mask conservation implies output mask = input masks (transpose relation)
            eq_word(m, V2m[j], V1m[j], name=f"xor_in1_{i}_{j}")
            eq_word(m, V2m[j], rotr(RrotB[j2], ROT_B[j] % nbits), name=f"xor_in2_{i}_{j}")

        # --- Fan-out split for V2m[j] because it is used in (t) and (FO) ---
        V2t  = [bits_from_word(m, f"V2t_{i}_{j}",  nbits) for j in range(lanes)]
        V2fo = [bits_from_word(m, f"V2fo_{i}_{j}", nbits) for j in range(lanes)]
        for j in range(lanes):
            fanout_conservation_word(m, V2m[j], [V2t[j], V2fo[j]], name=f"fan_V2_{i}_{j}")

        # 4) t = XOR over lanes of V2t[j]
        tm = bits_from_word(m, f"tm_{i}", nbits)
        for j in range(lanes):
            eq_word(m, V2t[j], tm, name=f"t_rel_{i}_{j}")

        # --- Fan-out split for t because it is used in every lane's FO ---
        tuse = [bits_from_word(m, f"tuse_{i}_{j}", nbits) for j in range(lanes)]
        fanout_conservation_word(m, tm, tuse, name=f"fan_t_{i}")

        # 5) FO[j] = V2fo[j] XOR ROTL(tuse[j], ROT_G[j])
        FOm = [bits_from_word(m, f"FOm_{i}_{j}", nbits) for j in range(lanes)]
        for j in range(lanes):
            eq_word(m, FOm[j], V2fo[j], name=f"fo_in1_{i}_{j}")
            eq_word(m, FOm[j], rotl(tuse[j], ROT_G[j] % nbits), name=f"fo_in2_{i}_{j}")

        # 6) R_{i+1} = L_i XOR FO
        for j in range(lanes):
            eq_word(m, Rm[j][i+1], Lm[j][i], name=f"Rnext_in1_{i}_{j}")
            eq_word(m, Rm[j][i+1], FOm[j],  name=f"Rnext_in2_{i}_{j}")

    # Pins (optional)
    if pins:
        for key, val in pins.items():
            mo = re.fullmatch(r'([LR])(\d+)_(\d+)', key)
            if not mo:
                raise ValueError(f"Bad pin key: {key}")
            side, jj, rnd = mo.group(1), int(mo.group(2)), int(mo.group(3))
            word = Lm[jj][rnd] if side == 'L' else Rm[jj][rnd]
            for b in range(nbits):
                bit = (val >> b) & 1
                word[b].LB = bit
                word[b].UB = bit

    # Non-trivial input mask (keep ONLY this, to avoid reviewer-attackable restrictions)
    m.addConstr(
        gp.quicksum(Lm[j][0][b] for j in range(lanes) for b in range(nbits)) +
        gp.quicksum(Rm[j][0][b] for j in range(lanes) for b in range(nbits)) >= 1,
        name="nontrivial_input_mask"
    )

    W = gp.quicksum(W_terms)
    m.setObjective(W, gp.GRB.MINIMIZE)
    return m, W


In [9]:
import math
from typing import Any, Dict, List, Optional, Tuple
import gurobipy as gp

def status_name(st: int) -> str:
    return {
        gp.GRB.OPTIMAL: "OPTIMAL",
        gp.GRB.INFEASIBLE: "INFEASIBLE",
        gp.GRB.TIME_LIMIT: "TIME_LIMIT",
        gp.GRB.INTERRUPTED: "INTERRUPTED",
        gp.GRB.INF_OR_UNBD: "INF_OR_UNBD",
    }.get(st, f"STATUS_{st}")

def default_Ts_linear() -> List[int]:
    """
    Candidate thresholds T for the feasibility query: “exists trail with W <= T?”
    You can tune this list: finer spacing at small T makes bracketing easier.
    """
    return [0, 8, 16, 24, 32, 40, 48, 56, 64, 80, 96, 112, 120, 124, 126, 127]

def refine_bracket_binary(
    env: gp.Env,
    rounds: int,
    nbits: int,
    lb_infeas: int,
    ub_feas: int,
    lanes: int,
    timelimit: int,
    threads: int,
) -> Tuple[int, int]:
    """
    Tighten a clean bracket [lb_infeas, ub_feas] where:
      - INFEASIBLE at T=lb_infeas  => proved minW > lb_infeas
      - FEASIBLE at T=ub_feas      => proved minW <= ub_feas
    If we hit TIME_LIMIT/UNKNOWN during refinement, we stop and return current bracket.
    """
    lo, hi = lb_infeas, ub_feas
    while lo + 1 < hi:
        mid = (lo + hi) // 2
        st, sols, rt = linear_exists_weight_le(
            env, rounds, nbits, mid, lanes, timelimit, threads, quiet=True
        )

        if st == gp.GRB.INFEASIBLE:
            lo = mid
        elif sols > 0:
            hi = mid
        else:
            # TIME_LIMIT or other unknown => stop refinement
            break

    return lo, hi

def adaptive_sweep_linear_64(
    env: gp.Env,
    rounds_list: List[int] = None,  # default 2..8 below
    nbits: int = 64,
    lanes: int = 8,
    Ts: Optional[List[int]] = None,
    timelimit_per_r: Optional[Dict[int, int]] = None,
    threads: int = 2,
    max_timeouts_in_a_row: int = 2,
    refine: bool = True,
) -> List[Dict[str, Any]]:

    if rounds_list is None:
        rounds_list = list(range(2, 9))  # 2..8

    if Ts is None:
        Ts = default_Ts_linear()

    # Ensure monotone increasing thresholds (important!)
    Ts = sorted(set(Ts))

    if timelimit_per_r is None:
        timelimit_per_r = {2: 3600, 3: 31200, 4: 31800, 5: 33600, 6: 33600, 7: 33600, 8: 33600}

    results: List[Dict[str, Any]] = []

    for r in rounds_list:
        tl = timelimit_per_r.get(r, 1800)
        print(f"\n=== r={r}, nbits={nbits}, timelimit={tl}s ===")

        last_infeas_T: Optional[int] = None   # proved minW > T
        first_feas_T: Optional[int] = None    # proved minW <= T
        timeouts_row = 0

        for T in Ts:
            st, sols, rt = linear_exists_weight_le(
                env, rounds=r, nbits=nbits, T=T,
                lanes=lanes, timelimit=tl, threads=threads, quiet=True
            )

            if st == gp.GRB.INFEASIBLE:
                verdict = "PROVED minW > T"
                last_infeas_T = T
                timeouts_row = 0

            elif sols > 0:
                # Even if status is TIME_LIMIT, SolCount>0 means we found a feasible trail.
                verdict = "FOUND trail with W<=T"
                first_feas_T = T
                timeouts_row = 0

            elif st == gp.GRB.TIME_LIMIT:
                verdict = "UNKNOWN (timeout)"
                timeouts_row += 1

            else:
                verdict = "OTHER/UNKNOWN"
                timeouts_row = 0

            print(f"  T={T:3d}  status={st}({status_name(st)})  sols={sols}  rt={rt:8.1f}s  => {verdict}")

            results.append({
                "rounds": r, "nbits": nbits, "T": T,
                "status": int(st), "status_name": status_name(st),
                "solcount": int(sols), "runtime_s": float(rt),
                "verdict": verdict,
            })

            # Monotonicity: once feasible at some T, no need to test larger T
            if first_feas_T is not None:
                break

            # If we keep timing out repeatedly, stop pushing further thresholds
            if timeouts_row >= max_timeouts_in_a_row:
                break

        # Print a per-round summary bracket (this is VERY paper-friendly)
        if (last_infeas_T is not None) and (first_feas_T is not None):
            lo, hi = last_infeas_T, first_feas_T
            if refine and (lo + 1 < hi):
                lo, hi = refine_bracket_binary(env, r, nbits, lo, hi, lanes, tl, threads)
                print(f"  Refined bracket: PROVED minW > {lo}, FOUND minW <= {hi}")
            else:
                print(f"  Bracket: PROVED minW > {lo}, FOUND minW <= {hi}")

        elif last_infeas_T is not None:
            print(f"  Lower bound only: PROVED minW > {last_infeas_T} (no feasible T found in sweep)")

        elif first_feas_T is not None:
            print(f"  Upper bound only: FOUND minW <= {first_feas_T} (no infeasible T proved in sweep)")

        else:
            print("  No bracket: all tested thresholds timed out or were unknown.")

    return results


In [12]:
import gurobipy as gp
from typing import Optional, List, Tuple, Dict, Any

def linear_exists_weight_le(
    env: gp.Env,
    rounds: int,
    nbits: int,
    T: int,
    lanes: int = 8,
    timelimit: int = 600,
    threads: int = 2,
    quiet: bool = True,
) -> Tuple[int, int, float]:
    """
    Feasibility: is there a linear trail with total weight W <= T ?
    Returns: (status, solcount, runtime_seconds)
    status: 3=INFEASIBLE (proved), 2=OPTIMAL with sol (found), 9=TIME_LIMIT (unknown)
    """
    m, W = build_fha_linear_milp(
        rounds=rounds,
        nbits=nbits,
        lanes=lanes,
        env=env,
        timelimit=timelimit,
        threads=threads
    )

    m.addConstr(W <= T, name="W_le_T")
    m.setObjective(0, gp.GRB.MINIMIZE)

    # Good defaults for proving infeasibility
    m.Params.MIPFocus = 2
    m.Params.Presolve = 2
    m.Params.Cuts = 2
    m.Params.Heuristics = 0.0

    if quiet:
        m.Params.OutputFlag = 0

    m.optimize()
    return m.Status, m.SolCount, float(m.Runtime)





res = adaptive_sweep_linear_64(
    env,
    rounds_list=list(range(2, 9)),
    nbits=64,
    lanes=8,
    Ts=[0, 8, 16, 24, 32, 40, 48, 64, 80, 96, 112, 120, 124, 126, 127],
    timelimit_per_r={2:3600, 3:31200, 4:14400, 5:14400, 6:14400, 7:14400, 8:14400},
    threads=2,
    max_timeouts_in_a_row=1,   # stop early if it’s clearly stalling
    refine=True
)



=== r=2, nbits=64, timelimit=3600s ===
Set parameter Threads to value 2
Set parameter TimeLimit to value 3600
Set parameter MIPFocus to value 2
Set parameter Presolve to value 2
Set parameter Cuts to value 2
Set parameter Heuristics to value 0

Interrupt request received
  T=  0  status=11(INTERRUPTED)  sols=0  rt=    17.1s  => OTHER/UNKNOWN
Set parameter Threads to value 2
Set parameter TimeLimit to value 3600
Set parameter MIPFocus to value 2
Set parameter Presolve to value 2
Set parameter Cuts to value 2
Set parameter Heuristics to value 0
  T=  8  status=2(OPTIMAL)  sols=1  rt=     1.8s  => FOUND trail with W<=T
  Upper bound only: FOUND minW <= 8 (no infeasible T proved in sweep)

=== r=3, nbits=64, timelimit=31200s ===
Set parameter Threads to value 2
Set parameter TimeLimit to value 31200
Set parameter MIPFocus to value 2
Set parameter Presolve to value 2
Set parameter Cuts to value 2
Set parameter Heuristics to value 0
